In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, year, month, dayofmonth, to_timestamp,from_unixtime,concat_ws,input_file_name
from pyspark.sql.types import ArrayType, StringType,TimestampType
from pyspark.sql import functions as F
from datetime import datetime
import re

In [29]:
GITHUB_BLACKLIST = {
    "github", "git", "bash", "shell", "sh",
    "html", "css", "windows", "flow"
}
keywords={
    'ember.js', 'trello', 'symfony', 'laravel', 'javascript', 'ansible', 'zoom', 'monday.com', 'lisp', 'asp.net core', 'rust', 'groovy', 'terminal', 'asp.net', 'excel', 'objective-c', 'angular', 'C', 'bigquery', 'microstrategy', 'react.js', 'dplyr', 'nuxt.js', 'deno', 'crystal', 'unify', 'atlassian', 'sql server', 'python', 'ssis', 'sap', 'f#', 'ionic', 'vue.js', 'digitalocean', 'redis', 'ocaml', 'nosql', 'shell', 'word', 'gtx', 'clojure', 'scala', 'elasticsearch', 'powerpoint', 'phoenix', 'git', 'unix', 'homebrew', 'dax', 'php', 'vmware', 'elixir', 'suse', 'centos', 'wrike', 'puppet', 'dynamodb', 'couchbase', 'kali', 'smartsheet', 'hugging face', 'mlpack', 'dart', 'pulumi', 'rshiny', 'fastify', 'unreal', 'vue', 'visual basic', 'nuix', 'fastapi', 'kubernetes', 'matlab', 'bitbucket', 'perl', 'keras', 'nltk', 'mattermost', 'sas', 'sql', 'lua', 'capacitor', 'flow', 'airtable', 'apl', 'graphql', 'ubuntu', 'yarn', 'node.js', 'redhat', 'fortran', 'arch', 'notion', 'qt', 'hadoop', 'opencv', 'theano', 'matplotlib', 'db2', 'assembly', 'kafka', 'scikit-learn', 'cassandra', 'gatsby', 'wimi', 'debian', 'mariadb', 'pyspark', 'visualbasic', 'svn', 'no-sql', 'asana', 'mxnet', 'chainer', 'linode', 'workfront', 'gcp', 'sqlserver', 'jquery', 'redshift', 'delphi', 'twilio', 'julia', 'swift', 'unity', 'neo4j', 'spark', 'vba', 'openstack', 'java', 'docker', 'play framework', 'esquisse', 'rocketchat', 'xamarin', 'ssrs', 'haskell', 'next.js', 'R', 'confluence', 'asp.netcore', 'planner', 'qlik', 'terraform', 'bash', 'powershell', 'ggplot2', 'dlib', 'electron', 'ruby on rails', 'blazor', 'angular.js', 'golang', 'spreadsheet', 'ibm cloud', 'gdpr', 'react', 'tidyr', 'microsoft lists', 'mongo', 'ringcentral', 'wire', 'spring', 'flask', 'aws', 'npm', 'c#', 'html', 'css', 'clickup', 'datarobot', 'cobol', 'ovh', 'couchdb', 'shogun', 'splunk', 'sass', 'flutter', 't-sql', 'codecommit', 'symphony', 'chef', 'rubyon rails', 'mlr', 'microsoft teams', 'ruby', 'linux', 'dingtalk', 'pascal', 'msaccess', 'looker', 'gitlab', 'drupal', 'airflow', 'seaborn', 'wsl', 'firestore', 'cognos', 'c++', 'fedora', 'sheets', 'numpy', 'plotly', 'github', 'power bi', 'colocation', 'typescript', 'sharepoint', 'mysql', 'svelte', 'jira', 'heroku', 'postgresql', 'sqlite', 'azure', 'vb.net', 'webex', 'tableau', 'databricks', 'kotlin', 'erlang', 'node', 'pytorch', 'alteryx', 'django', 'solidity', 'mongodb', 'outlook', 'tidyverse', 'windows', 'ms access', 'snowflake', 'selenium', 'slack', 'powerbi', 'cordova', 'huggingface', 'watson', 'jupyter', 'aurora', 'firebase', 'visio', 'express', 'oracle', 'google chat', 'spss', 'macos', 'jenkins', 'tensorflow', 'pandas'}

In [30]:

def extract_tools_udf(text):
    found = []
    if text:
        for kw in keywords:
            if len(kw) <= 2:
                pattern = r'\b' + re.escape(kw) + r'\b'
                if re.search(pattern, text):
                    found.append(kw)
            else:
                pattern = r'\b' + re.escape(kw.lower()) + r'\b'
                if re.search(pattern, text.lower()):
                    found.append(kw)
    return found

extract_tools_spark = udf(extract_tools_udf, ArrayType(StringType()))    

In [31]:
JOB_MAPPING = {
    "dataanalysis": "Data Analyst",
    "data-analysis": "Data Analyst",
    "datascience": "Data Scientist",
    "data-science": "Data Scientist",
    "dataengineering": "Data Engineer",
    "data-engineering": "Data Engineer",
    "businessanalysis": "Business Analyst",
    "business-analysis": "Business Analyst",
    "machinelearning": "Machine Learning Engineer",
    "machine-learning": "Machine Learning Engineer",
    "MachineLearning": "Machine Learning Engineer",
    "Software Engineer": "Software Engineer",
    "software engineer": "Software Engineer",
    "Cloud Engineer": "Cloud Engineer",
    "cloud-computing": "Cloud Engineer"
}

def normalize_job(job):
    if job is None:
        return "Unknown"
    key = job.lower().strip()
    return JOB_MAPPING.get(key, job)

normalize_job_udf = udf(normalize_job, StringType())

JOB_KEYS = set(JOB_MAPPING.keys())              
JOB_VALUES = set(JOB_MAPPING.values())   


In [32]:
def filter_blacklist_udf(tools):
    return [tool for tool in tools if tool.lower() not in GITHUB_BLACKLIST]
filter_udf = udf(filter_blacklist_udf, ArrayType(StringType()))    

def filter_stack_udf(tools):
    filtered = []
    for t in tools:
        if t:
            low = t.lower().strip()
            if low not in JOB_KEYS and low not in JOB_VALUES:
                filtered.append(t)
    return filtered

filter_stack = udf(filter_stack_udf, ArrayType(StringType()))    

In [33]:
spark = SparkSession.builder \
    .appName("SilverLayerProcessing") \
    .getOrCreate()

In [34]:
github_path = "abfss://bronze@datalakeyassin.dfs.core.windows.net/github/*.json"
stackoverflow_path = "abfss://bronze@datalakeyassin.dfs.core.windows.net/stackoverflow/*.json"
reddit_path = "abfss://bronze@datalakeyassin.dfs.core.windows.net/reddit/*.json"

silver_base_path = "abfss://silver@datalakeyassin.dfs.core.windows.net/"


df_github  = spark.read.option("multiline", "true").json(github_path)
df_stackoverflow  = spark.read.option("multiline", "true").json(stackoverflow_path)
df_reddit  = spark.read.option("multiline", "true").json(reddit_path)

In [35]:
control_file_path = silver_base_path + "_control/processed_files.txt"


processed_files_df = spark.read.text(control_file_path)
last_etl_time_str = processed_files_df.collect()[0][0].strip()
last_etl_time = datetime.strptime(last_etl_time_str, "%Y-%m-%d %H:%M:%S")

        
df_github = df_github.withColumn("filename", input_file_name())   
df_stackoverflow = df_stackoverflow.withColumn("filename", input_file_name())  
df_reddit = df_reddit.withColumn("filename", input_file_name())   

In [36]:
def extract_ts(filename):
    match = re.search(r'(\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2})', filename)
    if match:
        ts_str = match.group(1)
        return datetime.strptime(ts_str, "%Y-%m-%d_%H-%M-%S")
    return None

extract_ts_udf = udf(extract_ts, TimestampType())

In [37]:
df_github = df_github.withColumn("ingestion_time", extract_ts_udf(col("filename")))\
                   .drop("filename")
df_stackoverflow = df_stackoverflow.withColumn("ingestion_time", extract_ts_udf(col("filename")))\
                   .drop("filename")
df_reddit = df_reddit.withColumn("ingestion_time", extract_ts_udf(col("filename")))\
                   .drop("filename")

In [38]:
df_github = df_github.filter(col("ingestion_time") > last_etl_time)
df_stackoverflow = df_stackoverflow.filter(col("ingestion_time") > last_etl_time)
df_reddit = df_reddit.filter(col("ingestion_time") > last_etl_time)


In [39]:
df_github  = df_github.withColumn("source", F.lit("github"))
df_reddit  = df_reddit.withColumn("source", F.lit("reddit"))
df_stackoverflow   = df_stackoverflow.withColumn("source", F.lit("stackoverflow"))


In [40]:
df_github.show(5)
df_stackoverflow.show(5)
df_reddit.show(5)


In [41]:
df_github = df_github.withColumn("job", normalize_job_udf("job"))
df_reddit = df_reddit.withColumn("job", normalize_job_udf("job"))
df_stackoverflow = df_stackoverflow.withColumn("job", normalize_job_udf("job"))

In [42]:
df_reddit.select("job").distinct().show()


In [43]:
df_stackoverflow = df_stackoverflow.withColumn("date_converted", from_unixtime(col("date"))) \
                   .withColumn("year", year(col("date_converted"))) \
                   .withColumn("month", month(col("date_converted"))) \
                   .withColumn("day", dayofmonth(col("date_converted"))) \
                   .drop("date_converted")\
                   .drop("date")

# Reddit
df_reddit = df_reddit.withColumn("date_converted", from_unixtime(col("date"))) \
                     .withColumn("year", year(col("date_converted"))) \
                     .withColumn("month", month(col("date_converted"))) \
                     .withColumn("day", dayofmonth(col("date_converted"))) \
                     .drop("date_converted")\
                     .drop("date")

# GitHub
df_github = df_github.withColumn("created_at_dt", col("created_at").cast("timestamp")) \
                     .withColumn("year", year(col("created_at_dt"))) \
                     .withColumn("month", month(col("created_at_dt"))) \
                     .withColumn("day", dayofmonth(col("created_at_dt"))) \
                     .drop("created_at_dt")\
                     .drop("created_at")

In [44]:
df_github.select("year").distinct().show(5)


In [45]:
df_reddit = df_reddit.withColumn("text_to_parse", concat_ws(" ", col("title"), col("selftext")))\
                      .drop("title")\
                      .drop("selftext")
                      
df_github = df_github.withColumn("text_to_parse", concat_ws(" ", col("description"), col("readme")))\
                      .drop("description")\
                      .drop("readme")
df_stackoverflow = df_stackoverflow.withColumn("text_to_parse", concat_ws(" ", col("title"), col("body")))\
                      .drop("title")\
                      .drop("body")

In [46]:
df_github = df_github.withColumn("tools", extract_tools_spark(col("text_to_parse")))
df_github = df_github.withColumn("tools", filter_udf(col("tools")))
df_reddit=df_reddit.withColumn("tools", extract_tools_spark(col("text_to_parse")))
df_stackoverflow=df_stackoverflow.withColumn("tools", filter_stack(col("tools")))



In [47]:
df_github = df_github.drop("text_to_parse")\
                      .drop("url")\
                      .drop("language")


In [48]:
df_github.show(5)
df_reddit.show(5)
df_stackoverflow.show(5)

In [52]:


github_silver_path = silver_base_path + "github/"
stackoverflow_silver_path = silver_base_path + "stackoverflow/"
reddit_silver_path = silver_base_path + "reddit/"

df_github.write.mode("append").partitionBy("year", "month").parquet(github_silver_path)
df_stackoverflow.write.mode("append").partitionBy("year", "month").parquet(stackoverflow_silver_path)
df_reddit.write.mode("append").partitionBy("year", "month").parquet(reddit_silver_path)

In [50]:
# current_etl_time = datetime.utcnow()
# with open(control_file_path, "w") as f:
#     f.write(current_etl_time.strftime("%Y-%m-%d %H:%M:%S"))